# Introduction
This is details of a VR-PIPE setup of full calling pipeline of "reads" created by splitting PacBio reference genomes in 50kb chunks (don't know exact details - would need to get this from Thomas).

In [1]:
%run _standard_imports.ipynb
%run _shared_setup.ipynb


python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
DATA_DIR

'/nfs/team112_internal/rp7/data/pf3k'

In [11]:
RELEASE_DIR = "%s/pacbio_2" % DATA_DIR
RESOURCES_DIR = '%s/resources' % RELEASE_DIR

# GENOME_FN = "/nfs/pathogen003/tdo/Pfalciparum/3D7/Reference/Oct2011/Pf3D7_v3.fasta" # Note this ref used by Thomas is different to other refs we have used, e.g. chromsomes aren't in numerical order
GENOME_FN = "/lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta"
SNPEFF_DIR = "/lustre/scratch109/malaria/pf3k_methods/resources/snpEff"
REGIONS_FN = "/nfs/team112_internal/rp7/src/github/malariagen/pf-crosses/meta/regions-20130225.bed.gz"

RELEASE_METADATA_FN = "%s/pf3k_pacbio_2_sample_metadata.txt" % RELEASE_DIR
WG_VCF_FN = "%s/vcf/pf3k_pacbio_2.vcf.gz" % RELEASE_DIR

BCFTOOLS = '/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/pf3k_techbm/opt_4/bcftools/bcftools'
PICARD = 'java -jar /nfs/team112_internal/rp7/src/github/malariagen/methods-dev/pf3k_techbm/opt_4/picard/picard-tools-1.137/picard.jar'

VRPIPE_FOFN = "%s/pf3k_pacbio_2.fofn" % RELEASE_DIR
HC_INPUT_FOFN = "%s/pf3k_pacbio_2_hc_input.fofn" % RELEASE_DIR


In [5]:
print(WG_VCF_FN)

/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/pf3k_pacbio_2.vcf.gz


In [6]:
chromosomes = ["Pf3D7_%02d_v3" % x for x in range(1, 15, 1)]
#     'Pf3D7_API_v3', 'Pf_M76611'
# ]
chromosome_vcfs = ["%s/vcf/SNP_INDEL_%s.combined.filtered.vcf.gz" % (RELEASE_DIR, x) for x in chromosomes]
chromosome_vcfs

['/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_01_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_02_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_03_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_04_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_05_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_06_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_07_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_08_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_09_v3.combined.filtered.vcf.gz',
 '/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/SNP_INDEL_Pf3D7_10_v3.combined.filtered.vcf.gz',


In [7]:
if not os.path.exists(RESOURCES_DIR):
    os.makedirs(RESOURCES_DIR)

In [13]:
for lustre_dir in ['temp', 'input', 'output', 'meta']:
    new_dir = "/lustre/scratch109/malaria/pf3k_pacbio/%s" % lustre_dir
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

In [18]:
thomas_bam_sample_ids = ['Pf3D7II', 'Pf7G8', 'PfCD01', 'PfDd2', 'PfGA01', 'PfGB4', 'PfGN01', 'PfHB3',
                         'PfIT', 'PfKE01', 'PfKH01', 'PfKH02', 'PfML01', 'PfSD01', 'PfSN01', 'PfTG01']

In [12]:
sample_chrom = 'PfSD01_04_07'
sample_id = 'PfSD01'
!{PICARD} AddOrReplaceReadGroups I=/lustre/scratch108/parasites/tdo/Pfalciparum/PF3K/Reference12Genomes/Mapping_wholeChromosomes/ReRun_Splitting_50k_14072016/Res.Mapped.PfSD01_04_07.fasta.Merged.0407.bam \
O=/lustre/scratch109/malaria/pf3k_pacbio/temp/{sample_chrom}.bam RGID={sample_id} RGSM={sample_id} \
RGLB={sample_id} RGPU={sample_id} RGPL=illumina



[Mon Jul 18 18:24:30 BST 2016] picard.sam.AddOrReplaceReadGroups INPUT=/lustre/scratch108/parasites/tdo/Pfalciparum/PF3K/Reference12Genomes/Mapping_wholeChromosomes/ReRun_Splitting_50k_14072016/Res.Mapped.PfSD01_04_07.fasta.Merged.0407.bam OUTPUT=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_04_07.bam RGID=PfSD01 RGLB=PfSD01 RGPL=illumina RGPU=PfSD01 RGSM=PfSD01    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Mon Jul 18 18:24:30 BST 2016] Executing as rp7@malsrv3 on Linux 3.2.0-75-generic amd64; OpenJDK 64-Bit Server VM 1.6.0_32-b32; Picard version: 1.137(c8c9f584d7042e98867ea3cfc355fb7b89acd2e6_1438018139) IntelDeflater
INFO	2016-07-18 18:24:31	AddOrReplaceReadGroups	Created read group ID=PfSD01 PL=illumina LB=PfSD01 SM=PfSD01

[Mon Jul 18 18:24:33 BST 2016] picard.sam.AddOrReplaceReadGroups done. Elapsed time: 0.06 minutes.
Runtime.totalMemo

In [36]:
sample_id = 'PfSD01'
sample_chrom = 'PfSD01_04_07'
input_bams = ['I=/lustre/scratch109/malaria/pf3k_pacbio/temp/%s.bam' % sample_chrom] + ['I=/lustre/scratch109/malaria/pf3k_pacbio/temp/%s_%02d.bam' % (sample_id, x) for x in
              [1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14]]
output_bam = '/lustre/scratch109/malaria/pf3k_pacbio/input/%s.bam' % sample_id
!{PICARD} MergeSamFiles {' '.join(input_bams)} O={output_bam} MERGE_SEQUENCE_DICTIONARIES=true
!{PICARD} BuildBamIndex I={output_bam}

[Mon Jul 18 21:01:49 BST 2016] picard.sam.MergeSamFiles INPUT=[/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_04_07.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_01.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_02.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_03.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_05.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_06.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_08.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_09.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_10.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_11.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_12.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_13.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_14.bam] OUTPUT=/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01.bam MERGE_SEQUENCE_DICTIONARIES=true    SORT_ORDER=coordinate ASSUME_SORTED=false USE_THREADING=false VERBOS

In [43]:
!{PICARD} ReplaceSamHeader I={output_bam} O={output_bam.replace('.bam', 'reheader.bam')} \
HEADER=/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01.header.sam

[Mon Jul 18 21:26:45 BST 2016] picard.sam.ReplaceSamHeader INPUT=/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01.bam HEADER=/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01.header.sam OUTPUT=/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01reheader.bam    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Mon Jul 18 21:26:45 BST 2016] Executing as rp7@malsrv3 on Linux 3.2.0-75-generic amd64; OpenJDK 64-Bit Server VM 1.6.0_32-b32; Picard version: 1.137(c8c9f584d7042e98867ea3cfc355fb7b89acd2e6_1438018139) IntelDeflater
[Mon Jul 18 21:26:46 BST 2016] picard.sam.ReplaceSamHeader done. Elapsed time: 0.01 minutes.
Runtime.totalMemory()=125435904


In [37]:
input_bams

['I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_04_07.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_01.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_02.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_03.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_05.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_06.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_08.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_09.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_10.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_11.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_12.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_13.bam',
 'I=/lustre/scratch109/malaria/pf3k_pacbio/temp/PfSD01_14.bam']

In [45]:
sample_id = 'PfSD01'
output_bam = '/lustre/scratch109/malaria/pf3k_pacbio/input/%s.bam' % sample_id
!java -jar /nfs/team112_internal/rp7/src/github/malariagen/methods-dev/pf3k_techbm/opt/picard/picard-tools-1.135/picard.jar \
ReorderSam I={output_bam.replace('.bam', 'reheader.bam')} O={output_bam.replace('.bam', '.sorted.bam')} \
REFERENCE=/lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta
# CREATE_INDEX=TRUE
!mv {output_bam.replace('.bam', '.sorted.bam')} {output_bam}
!{PICARD} BuildBamIndex I={output_bam}

[Mon Jul 18 21:28:31 BST 2016] picard.sam.ReorderSam INPUT=/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01reheader.bam OUTPUT=/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01.sorted.bam REFERENCE=/lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta    ALLOW_INCOMPLETE_DICT_CONCORDANCE=false ALLOW_CONTIG_LENGTH_DISCORDANCE=false VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Mon Jul 18 21:28:31 BST 2016] Executing as rp7@malsrv3 on Linux 3.2.0-75-generic amd64; OpenJDK 64-Bit Server VM 1.6.0_32-b32; Picard version: 1.135(83ec44e03ec8d07ef180ef86b7a62d59a80dedd3_1435607037) IntelDeflater
INFO	2016-07-18 21:28:31	ReorderSam	SAM/BAM file
INFO	2016-07-18 21:28:31	ReorderSam	  SN=%s LN=%d%nPf3D7_04_v31200490
INFO	2016-07-18 21:28:31	ReorderSam	  SN=%s LN=%d%nPf3D7_07_v31445207
INFO	2016-07-18 21:28:31	ReorderSam	  SN=%s LN=%d%n

In [29]:
output_bam

'/lustre/scratch109/malaria/pf3k_pacbio/input/PfSD01.bam'

In [46]:
# I messed up the folowing by accident so had to recreate
for sample_id in ['PfTG01']:
    input_bams = ['I=/lustre/scratch109/malaria/pf3k_pacbio/temp/%s_%02d.bam' % (sample_id, x) for x in range(1, 15)]
    output_bam = '/lustre/scratch109/malaria/pf3k_pacbio/input/%s.bam' % sample_id
    !{PICARD} MergeSamFiles {' '.join(input_bams)} O={output_bam} MERGE_SEQUENCE_DICTIONARIES=true
    !{PICARD} BuildBamIndex I={output_bam}

[Mon Jul 18 21:30:55 BST 2016] picard.sam.MergeSamFiles INPUT=[/lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_01.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_02.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_03.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_04.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_05.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_06.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_07.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_08.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_09.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_10.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_11.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_12.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_13.bam, /lustre/scratch109/malaria/pf3k_pacbio/temp/PfTG01_14.bam] OUTPUT=/lustre/scratch109/malaria/pf3k_pacbio/input/PfTG01.bam MERGE_SEQUENCE_DICTIONARIES=true    SORT_ORDER=c

In [40]:
# # Removed PfSD01 as no reads for chromsome 4 - see email from Thomas 14/07/2016 20:17 and response 17/07/2016 11:37
# thomas_bam_sample_ids = ['Pf3D7II', 'Pf7G8', 'PfCD01', 'PfDd2', 'PfGA01', 'PfGB4', 'PfGN01', 'PfHB3',
#                          'PfIT', 'PfKE01', 'PfKH01', 'PfKH02', 'PfML01', 'PfSN01', 'PfTG01']

In [20]:
fo = open(HC_INPUT_FOFN, 'w')
for sample_id in thomas_bam_sample_ids:
    output_bam = '/lustre/scratch109/malaria/pf3k_pacbio/input/%s.bam' % sample_id
    print('%s' % (output_bam), file=fo)
fo.close()


In [19]:
HC_INPUT_FOFN

'/nfs/team112_internal/rp7/data/pf3k/pacbio_2/pf3k_pacbio_2_hc_input.fofn'

# VR-PIPE pipeline
These were created using vrpipe-setup --based_on, with the setups originally created in 20160716_pf3k_pacbio_1_setup

# Create whole genome VCF

In [51]:
!{BCFTOOLS} concat {" ".join(chromosome_vcfs)} \
| sed 's/##FORMAT=<ID=AD,Number=./##FORMAT=<ID=AD,Number=R/' \
| bgzip -c > {WG_VCF_FN}
!tabix -p vcf {WG_VCF_FN}

In [48]:
WG_VCF_FN

'/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/pf3k_pacbio_2.vcf.gz'

# Create README section containing variant numbers

In [52]:
number_of_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'TYPE="snp"' {WG_VCF_FN} | wc -l
number_of_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'TYPE!="snp"' {WG_VCF_FN} | wc -l
number_of_pass_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp"' {WG_VCF_FN} | wc -l
number_of_pass_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp"' {WG_VCF_FN} | wc -l
number_of_pass_biallelic_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp" && N_ALT=1' {WG_VCF_FN} | wc -l
number_of_pass_biallelic_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp" && N_ALT=1' {WG_VCF_FN} | wc -l
number_of_VQSLODgt6_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp" && VQSLOD>6' {WG_VCF_FN} | wc -l
number_of_VQSLODgt6_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp" && VQSLOD>6' {WG_VCF_FN} | wc -l

print("%s variants" % ("{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0]))))
print("%s SNPs" % ("{:,}".format(int(number_of_snps[0]))))
print("%s indels" % ("{:,}".format(int(number_of_indels[0]))))
print()
print("%s PASS variants" % ("{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0]))))
print("%s PASS SNPs" % ("{:,}".format(int(number_of_pass_snps[0]))))
print("%s PASS indels" % ("{:,}".format(int(number_of_pass_indels[0]))))
print()
print("%s PASS biallelic variants" % ("{:,}".format(int(number_of_pass_biallelic_snps[0]) + int(number_of_pass_biallelic_indels[0]))))
print("%s PASS biallelic SNPs" % ("{:,}".format(int(number_of_pass_biallelic_snps[0]))))
print("%s PASS biallelic indels" % ("{:,}".format(int(number_of_pass_biallelic_indels[0]))))
print()
print("%s VQSLOD>6.0 variants" % ("{:,}".format(int(number_of_VQSLODgt6_snps[0]) + int(number_of_VQSLODgt6_indels[0]))))
print("%s VQSLOD>6.0 SNPs" % ("{:,}".format(int(number_of_VQSLODgt6_snps[0]))))
print("%s VQSLOD>6.0 indels" % ("{:,}".format(int(number_of_VQSLODgt6_indels[0]))))
print()

259,931 variants
104,118 SNPs
155,813 indels

221,850 PASS variants
79,776 PASS SNPs
142,074 PASS indels

127,070 PASS biallelic variants
67,297 PASS biallelic SNPs
59,773 PASS biallelic indels

176,843 VQSLOD>6.0 variants
61,191 VQSLOD>6.0 SNPs
115,652 VQSLOD>6.0 indels



In [53]:
"{number_of_pass_variants}/{number_of_variants} variants ({pct_pass}%) pass all filters".format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        pct_pass=round((
            (int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])) /
            (int(number_of_snps[0]) + int(number_of_indels[0]))
        ) * 100)
)

'221,850/259,931 variants (85%) pass all filters'

In [52]:
"{number_of_pass_variants}/{number_of_variants} variants ({pct_pass}%) pass all filters".format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        pct_pass=round((
            (int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])) /
            (int(number_of_snps[0]) + int(number_of_indels[0]))
        ) * 100)
)

'214,349/251,945 variants (85%) pass all filters'

In [54]:
print('''
The VCF file contains details of {number_of_variants} discovered variants of which {number_of_snps}
are SNPs and {number_of_indels} are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are {number_of_pass_variants} such high-
quality PASS variants ({number_of_pass_snps} SNPs and {number_of_pass_indels} indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a quality score called VQSLOD. All
variants with a VQSLOD score below 0 are filtered out, i.e. will have a value of
Low_VQSLOD in the FILTER column, rather than PASS. The VQSLOD score for each
variant can be found in the INFO field of the VCF file. It is possible to use the
VQSLOD score to define a more or less stringent set of variants. For example for
a very stringent set of the highest quality variants, select only those variants
where VQSLOD >= 6. There are {number_of_VQSLODgt6_snps} such stringent SNPs and {number_of_VQSLODgt6_indels}
such stringent indels.

It is also important to note that some variants have more than two alleles. For
example, amongst the {number_of_pass_snps} high quality PASS SNPs, {number_of_pass_biallelic_snps} are biallelic. The
remaining {number_of_pass_multiallelic_snps} high quality PASS SNPs have 3 or more alleles. Similarly, amongst
the {number_of_pass_indels} high-quality PASS indels, {number_of_pass_biallelic_indels} are biallelic. The remaining
{number_of_pass_multiallelic_indels} high quality PASS indels have 3 or more alleles.
'''.format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_snps="{:,}".format(int(number_of_snps[0])),
        number_of_indels="{:,}".format(int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        number_of_pass_snps="{:,}".format(int(number_of_pass_snps[0])),
        number_of_pass_indels="{:,}".format(int(number_of_pass_indels[0])),
        number_of_VQSLODgt6_snps="{:,}".format(int(number_of_VQSLODgt6_snps[0])),
        number_of_VQSLODgt6_indels="{:,}".format(int(number_of_VQSLODgt6_indels[0])),
        number_of_pass_biallelic_snps="{:,}".format(int(number_of_pass_biallelic_snps[0])),
        number_of_pass_biallelic_indels="{:,}".format(int(number_of_pass_biallelic_indels[0])),
        number_of_pass_multiallelic_snps="{:,}".format(int(number_of_pass_snps[0]) - int(number_of_pass_biallelic_snps[0])),
        number_of_pass_multiallelic_indels="{:,}".format(int(number_of_pass_indels[0]) - int(number_of_pass_biallelic_indels[0])),
    )
)


The VCF file contains details of 259,931 discovered variants of which 104,118
are SNPs and 155,813 are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are 221,850 such high-
quality PASS variants (79,776 SNPs and 142,074 indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a qual

In [53]:
print('''
The VCF file contains details of {number_of_variants} discovered variants of which {number_of_snps}
are SNPs and {number_of_indels} are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are {number_of_pass_variants} such high-
quality PASS variants ({number_of_pass_snps} SNPs and {number_of_pass_indels} indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a quality score called VQSLOD. All
variants with a VQSLOD score below 0 are filtered out, i.e. will have a value of
Low_VQSLOD in the FILTER column, rather than PASS. The VQSLOD score for each
variant can be found in the INFO field of the VCF file. It is possible to use the
VQSLOD score to define a more or less stringent set of variants. For example for
a very stringent set of the highest quality variants, select only those variants
where VQSLOD >= 6. There are {number_of_VQSLODgt6_snps} such stringent SNPs and {number_of_VQSLODgt6_indels}
such stringent indels.

It is also important to note that some variants have more than two alleles. For
example, amongst the {number_of_pass_snps} high quality PASS SNPs, {number_of_pass_biallelic_snps} are biallelic. The
remaining {number_of_pass_multiallelic_snps} high quality PASS SNPs have 3 or more alleles. Similarly, amongst
the {number_of_pass_indels} high-quality PASS indels, {number_of_pass_biallelic_indels} are biallelic. The remaining
{number_of_pass_multiallelic_indels} high quality PASS indels have 3 or more alleles.
'''.format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_snps="{:,}".format(int(number_of_snps[0])),
        number_of_indels="{:,}".format(int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        number_of_pass_snps="{:,}".format(int(number_of_pass_snps[0])),
        number_of_pass_indels="{:,}".format(int(number_of_pass_indels[0])),
        number_of_VQSLODgt6_snps="{:,}".format(int(number_of_VQSLODgt6_snps[0])),
        number_of_VQSLODgt6_indels="{:,}".format(int(number_of_VQSLODgt6_indels[0])),
        number_of_pass_biallelic_snps="{:,}".format(int(number_of_pass_biallelic_snps[0])),
        number_of_pass_biallelic_indels="{:,}".format(int(number_of_pass_biallelic_indels[0])),
        number_of_pass_multiallelic_snps="{:,}".format(int(number_of_pass_snps[0]) - int(number_of_pass_biallelic_snps[0])),
        number_of_pass_multiallelic_indels="{:,}".format(int(number_of_pass_indels[0]) - int(number_of_pass_biallelic_indels[0])),
    )
)


The VCF file contains details of 251,945 discovered variants of which 100,477
are SNPs and 151,468 are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are 214,349 such high-
quality PASS variants (76,589 SNPs and 137,760 indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a qual

# Make all files read-only

In [23]:
# Seems like Jim prefers team112 to have write access, so haven't made unreadable
# !chmod -R uga-w {RELEASE_DIR}

chmod: changing permissions of `/nfs/team112_internal/rp7/data/pf3k/mrs_1/pf3k_mrs_1_thomas_bams.txt': Operation not permitted


In [54]:
RELEASE_DIR

'/nfs/team112_internal/rp7/data/pf3k/pacbio_1'

# Cleanup
vrpipe-setup --setup pf3k_pacbio_2_haplotype_caller --deactivate

vrpipe-setup --setup pf3k_pacbio_2_combine_gvcfs --deactivate

vrpipe-setup --setup pf3k_pacbio_2_genotype_gvcfs --deactivate

vrpipe-setup --setup pf3k_pacbio_2_variant_recalibration_snps_QD_SOR_DP --deactivate

vrpipe-setup --setup pf3k_pacbio_2_variant_recalibration_indels_QD_DP --deactivate

vrpipe-setup --setup pf3k_pacbio_2_apply_recalibration_snps --deactivate

vrpipe-setup --setup pf3k_pacbio_2_apply_recalibration_indels --deactivate

vrpipe-setup --setup pf3k_pacbio_2_annotate_snps --deactivate

vrpipe-setup --setup pf3k_pacbio_2_annotate_indels --deactivate

vrpipe-setup --setup pf3k_pacbio_2_combine_variants --deactivate

vrpipe-setup --setup pf3k_pacbio_2_variant_filtration --deactivate

